*** Veri Bilimi ve Makine Öğrenmesine Giriş Dönem Projesi:**
 
Mağaza müdürü vitrinlerinde tablet ürünlerini en doğru şekilde sergileyebilmek için 2.000 adet tableti fiyat/performans durumlarına göre “Ucuz”, “Normal”, “Pahalı”, “Çok Ucuz” olarak etiketlemiştir.

Mağazasına yeni gelecek tablet ürünlerini etiketleyerek vakit kaybetmek istemeyen IT ekibi mevcut verilerle başarılı bir makine öğrenmesi algoritması geliştirmek istemektedir. Böylelikle yeni gelecek tablet ürünlerini hangi vitrinde sergilemeleri gerektiğini hızlı ve güvenilir şekilde elde edebileceklerdir.

IT ekip lideri bu projeyi keşifçi veri analizi, ön işleme ve tahmine dayalı analitik makine öğrenmesi olmak üzere 3 safha olarak planlamaktadır.



# Veriyi Tanıma Ve Sindirme

**Öncelikle ihtiyacımız dogrultusunda kullanacagımız kütüphaneleri import edelim.**

In [ ]:
import pandas as pd
import numpy as np
import missingno                  
from sklearn import preprocessing 
import seaborn as sns
import re                        
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error , mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

**Tablet veri setimizi read_csv fonksiyonu sayesinde tablets değişkenine dahil ediyoruz sonrasında ise df değişkeni yaratıp tablets değişkenini kopyalıyoruz.**

In [ ]:
tablet = pd.read_csv('/kaggle/input/tablet-csv/tablet.csv')
df = tablet.copy()
df

# Kesifci Veri Analizi (Exploratory Data Analaysis)

**Veri setinin ilk 5 elemanını gözlemleyelim.**

In [ ]:
df.head()

**Veri setinin rastgele 6 elemanı gözlemleyim .**

In [ ]:
df.sample(6)

**Veri setinin sondaki 5 elemanını yazdıralım .**

In [ ]:
df.tail()

**Veri setinin nitelik ve özniteliklerini gözlemleyelim.**

In [ ]:
df.shape

**Veri setimiz hakkında temel bilgileri yazdırma**

burada veri setinin yapisal bilgilerini ogrenmek icin info fonksiyonu kullanildi
Her bir değişkenimizin kaçar adet olduğunu,değişkenlerin tipini ve bellek kullanımını görüntüledik

In [ ]:
df.info()


**Veri tiplerini görüntüleyelim**

In [ ]:
df.dtypes

**Hedef değişkenimizin dengeli dağılıp dağılmadığını sorgulayalım**

In [ ]:
df["Bluetooth"].value_counts()

In [ ]:
df["Dokunmatik"].value_counts()

In [ ]:
df["FiyatAraligi"].value_counts()

In [ ]:
df["Renk"].value_counts()

**Veri setimizin standart sapma, dağılımı, varyansı gibi nitelikleri belirten describe().T tablosunu yazdırma**

Veri çerçevesindeki sayısal değişkenler için temel istatistik değerleri

In [ ]:
df.describe().T


**Korelasyon matrisini yazdıralım**

In [ ]:
numeric_df = df.select_dtypes(include=['number'])  # Sadece sayısal sütunları seç
corr = numeric_df.corr()
corr

**Isı haritası çizelim**

In [ ]:
numeric_df = df.select_dtypes(include=['number'])  # Sadece sayısal sütunları seç
corr = numeric_df.corr()
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

* Tablonun tamamina yakını siyah renkte olduğundan öznitelikler arasindaki iliski yok denecek kadar azdır

In [ ]:
numeric_df = df.select_dtypes(include=['number'])  # Sadece sayısal sütunları seç
corr = numeric_df.corr()["ArkaKameraMP"]["OnKameraMP"]
corr

* Burada korelasyon iliskisi en yuksek olan iki ozniteligimiz arasindaki korelasyon oranini 0.6456 seklinde bulduk fakat burada da guclu bir iliski var diyemeyiz orta duzeyde bir iliski oldugu soylenebilir.

In [ ]:
numeric_df = df.select_dtypes(include=['number'])  # Sadece sayısal sütunları seç
corr = numeric_df.corr()["CozunurlukYükseklik"]["CozunurlukGenislik"]
corr

*  Burada aralarinda iliski olan bir diger CozunurlukYükseklik ve CozunurlukGenislik olan iki ozniteligimiz arasindaki korelasyon oranini 0.5106 seklinde bulduk fakat burada da guclu bir iliski vardir diyemeyiz orta duzeyde bir iliski oldugundan bahsedebiliriz.

**Veri çercevesinin özniteliklerini değerlendirelim**

Buradan FiyatAraligi degiskenimizin 4 adet benzersiz degiskeni oldugunu gormus olduk

In [ ]:
df["FiyatAraligi"].unique()


Dengeli dagilim oldugunu gozlemlemis olduk.

In [ ]:
df['FiyatAraligi'].value_counts()


In [ ]:
df["MikroislemciHizi"].unique()

In [ ]:
df["MikroislemciHizi"].nunique()

In [ ]:
df["MikroislemciHizi"].value_counts()

In [ ]:
df["CekirdekSayisi"].unique()

In [ ]:
df["CekirdekSayisi"].nunique()

In [ ]:
df["Kalinlik"].value_counts()

In [ ]:
df["CekirdekSayisi"].unique()

In [ ]:
df["CekirdekSayisi"].nunique()

In [ ]:
df["CekirdekSayisi"].value_counts()

# Görselleştirme 

**Veri setimizin özniteliklerini tablolar aracılığıyla görselleştirelim**

countplot, seaborn kütüphanesinde bulunan bir fonksiyondur ve kategorik verilerin sıklığını görselleştirmek için kullanılır. Genellikle bir veri setindeki farklı kategorik değerlerin kaç kez tekrarlandığını saymak ve bunları karşılaştırmak istediğinizde kullanılır.

In [ ]:
sns.countplot(x = "FiyatAraligi", data = df)

* Hepsinden esit sayida gozlem var dolayisiyla dengeli bir dagilim soz konusudur

displot -> dagilim gorsellestirmek icin kullanilan bir fonksiyon

In [ ]:
sns.distplot(df["ArkaKameraMP"], bins=16, color="green");


violinplot, veri dağılımlarını görselleştirmek için kullanılan bir grafik türüdür ve genellikle kategorik ve sürekli verilerin ilişkisini göstermek için tercih edilir. Bu grafik türü, veri dağılımlarını kutu plotlarına benzer bir şekilde gösterirken aynı zamanda kernel yoğunluk tahminini de içerir.

In [ ]:
sns.violinplot(y = "BataryaGucu", color = "red", data = df)

"histplot", seaborn kütüphanesinde bulunan bir fonksiyondur ve genellikle sürekli değişkenlerin dağılımını görselleştirmek için kullanılır. Bu fonksiyon, veri setindeki bir değişkenin histogramını çizmenizi sağlar.

In [ ]:
sns.histplot(df["MikroislemciHizi"], bins=16, color="purple");


In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "RAM", data = df);

* Buradaki grafigimizden FiyatAraligi pahali olanlarin daha fazla RAM degeri olacak oldugu sonucunu cikarabiliriz.

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "MikroislemciHizi", data = df);

In [ ]:
sns.distplot(df["DahiliBellek"], bins = 60, color = "orange")

barplot", seaborn kütüphanesinde bulunan bir fonksiyondur ve kategorik değişkenlerin ortalamasını veya toplamını görselleştirmek için kullanılır. Bu grafik türü, bir kategorik değişkenin farklı seviyelerinin (kategorilerinin) bir numarasal değişkene göre ortalama değerlerini veya toplam değerlerini gösterir.

In [ ]:
sns.barplot(x = "FiyatAraligi", y = "CekirdekSayisi", data = df);

* Buradaki grafigimizden FiyatAraligi ozniteligi ile CekirdekSayisi arasindaki iliskiyi goruntulemis olduk.

In [ ]:
sns.violinplot(y = "Kalinlik", color = "blue", data = df)

In [ ]:
sns.distplot(df["Agirlik"], bins = 120, color = "blue")

In [ ]:
sns.distplot(df["BataryaOmru"], bins = 15, color = "cyan")

"CozunurlukYükseklik" ile "CozunurlukGenislik" öznitelikleri arasında ilişkisel görselleştirme yapılması

"RAM" ile diğer bazı öznitelikler arasında ilişkisel görselleştirme yapılması

In [ ]:
sns.jointplot(x=df["CozunurlukYükseklik"], y=df["CozunurlukGenislik"], kind = "kde");

"scatterplot", veri setindeki iki sürekli değişken arasındaki ilişkiyi görselleştirmek için kullanılan bir grafik türüdür. Her bir nokta, veri setindeki bir gözlemi temsil eder ve x ve y eksenlerindeki konumu, ilgili iki değişkenin değerlerini gösterir.

In [ ]:
sns.scatterplot(x="RAM", y="CozunurlukYükseklik", hue="FiyatAraligi", data=df)

In [ ]:
sns.scatterplot(x="CozunurlukYükseklik", y="CozunurlukGenislik", data=df);

"OnKameraMP" ile "ArkaKameraMP" öznitelikleri arasında ilişkisel görselleştirme yapılması

In [ ]:
sns.jointplot(x="OnKameraMP", y="ArkaKameraMP", data=df);

**Kategorik değişkenler**

In [ ]:
sns.countplot(x = "4G", data = df)

In [ ]:
sns.countplot(x = "WiFi", data = df)

In [ ]:
sns.countplot(x = "CiftHat", data = df)

In [ ]:
sns.countplot(x = "3G", data = df)

In [ ]:
sns.countplot(x = "Renk", data = df)

In [ ]:
df.hist(figsize = (20,15))
plt.show()

In [ ]:
df

In [ ]:
df.shape

In [ ]:
len(df.index)

In [ ]:
df.columns

## Eksik Değerler (NA, NaN, Null, None, Missing Value etc.) 

### Eksik Değerlerin Tespiti

* **info()** fonksiyonu kullanarak tablodaki eksik verileri, bu verilerin column bilgilerini ve türlerini tespit edebiliriz.

In [ ]:
df.info()

* **isna().sum().sum()** fonksiyonlarını kullanrak veri çerçevesinde toplamda kaç eksik gözlem var? Sorusuna cevap buluruz.

In [ ]:
df.isna().sum().sum()

* **isna().sum()** fonksiyonlarını kullanarak veri çerçevesindeki eksik gözlemlerin hangi **değişkenlede(columns)** bulunduğunu ve o değişkenlerdeki eksik gözlemlerin kaç adet olduğunu buluruz.

In [ ]:
df.isna().sum()

### Eksik Verili Değişkenlerin İlişkisi 

**Eksik değerler barındıran 3 özniteliğimiz var.*

"Bu üç öznitelikte eksik değerler rassal şekilde mi meydana geldi yoksa bu değişkenlerin eksik değerleri arasında bir ilişki var mı?" sorusunu görselleştiriyoruz.

* **missingno.matrix(veri, tablo boyutu)** fonksiyonunu kullanarak verimizdeki eksik verilerin dağılımını görmek için görselleştirme yapıyoruz. 

In [ ]:
missingno.matrix(df, figsize=(12,8))

* **describe()** fonksiyonuyla veri çeçevemizin istatiksel özetini inceliyoruz.

In [ ]:
df.describe().T

### Eksik OnKameraMP Verilerini Doldurmak

**OnKameraMP** bilgisi eksik olan gözlemleri tüm öznitelikleriyle görüntüleyelim, göze çarpan bir şey var mı?

In [ ]:
df[df["OnKameraMP"].isna()]

* Yukarıdaki tabloyu incelediğimizde **OnKameraMP** bilgisi eksik olan gözlemlerin hepsinin **FiyatAraligi** değeri aynı ve **Çok Ucuz** dur. Ek olarak hepsininin **3G** değeri aynı ve **Var** dır. Bu çıkarımları ele alarak eksik değerleri doldurabiliriz.
* Aşağıdaki kullanım ile **FiyatAraligi** değeri **Çok Ucuz** ve **3G** değeri **Var** olan verilerin OnKameraMP değerlerinin ortalaması bulunur.

In [ ]:
df[(df["FiyatAraligi"]=="Çok Ucuz") & (df["3G"]=="Var")]["OnKameraMP"].mean()

* Bulunan ortalama değer bir değişkene atanır.

In [ ]:
valueOnKameraMP=df[(df["FiyatAraligi"]=="Çok Ucuz") & (df["3G"]=="Var")]["OnKameraMP"].mean() 

* **OnKameraMP** bilgisi eksik olan gözlemlerin hepsinin index değeri indexes değişkenine atanır. Böylece eksik verileri doldururken kullanılabilir.

In [ ]:
indexes=df[df["OnKameraMP"].isna()].index

* **OnKameraMP** bilgisi eksik olan gözlemlerin hepsini bulunan ortalama değerin yuvarlanmış değeriyle doldurduk.

In [ ]:
df.loc[indexes, "OnKameraMP"]= valueOnKameraMP

### Eksik RAM Verilerini Doldurmak

* **RAM** bilgisi eksik olan gözlemleri tüm öznitelikleriyle görüntüleyelim, göze çarpan bir şey var mı?

In [ ]:
df[df["RAM"].isna()]

* Yukarıdaki tabloyu incelediğimizde **RAM** bilgisi eksik olan gözlemlerin hepsinin **FiyatAraligi** değeri aynı ve **Pahalı** dır. Bu çıkarımı ele alarak eksik değerleri doldurabiliriz.
* Aşağıdaki kullanım ile **FiyatAraligi** değeri **Pahalı** olan verilerin RAM değerlerinin ortalaması bulunur. Bir değişkene atanır.

In [ ]:
valueRAM=df[(df["FiyatAraligi"]=="Pahalı")]["RAM"].mean()


* **RAM** bilgisi eksik olan gözlemlerin hepsinin index değeri indexes değişkenine atanır. Böylece eksik verileri doldururken kullanılabilir.

In [ ]:
indexes=df[df["RAM"].isna()].index

* Aşağıdaki kullanım ile **RAM** bilgisi eksik olan gözlemlerin hepsini bulunan ortalama değerin yuvarlanmış değeriyle doldurduk.

In [ ]:
df.loc[indexes,"RAM"]= valueRAM

### Eksik Veri Kalmadığını Teyit Etmek 

* **OnKameraMP** değişkenini tekrar kontrol ediyoruz. Eksik veriler var mı?

In [ ]:
df[df["OnKameraMP"].isna()]

* **RAM** değişkenini tekrar kontrol ediyoruz. Eksik veriler var mı?

In [ ]:
df[df["RAM"].isna()]

* info() fonksiyonu kullanarak tabloda eksik veri var mı? Kontrol ediyoruz.

In [ ]:
df.info()

* **isna().sum()** fonksiyonlarını kullanarak veri çerçevesinde eksik gözlemler var mı? Kontrol ediyoruz.

In [ ]:
df.isna().sum()

* Son olarak tüm veri çerçevesindeki toplam eksik veri sayısını kontrol ederiz. 

In [ ]:
df.isna().sum().sum()

* Yukarıda da görüldüğü üzere eksik verilerin izlenen yol ile doldurulduğu ve veri çerçevesinde herhangi bir eksik verinin kalmadığı görülür.

### Kategorik Değerleri Dönüştürme 

In [ ]:
df

#### Label / One-Hot Encoding

* **Bluetooth, CiftHat, 4G, 3G, Dokunmatik ve WiFi** değişkenlerinde bulunan "Var" ve "Yok" ifadeleri bilgisayarın anlayacağı formatta değildir. Dolayısıyla **label encoding** yaparak bu ifadeler için bilgisayarın anlayacağı *0, 1* dönüşümü yapalım.

* **label encoding** yapmak için bir label encoding değişkeni oluştururuz.

In [ ]:
label_encoder = preprocessing.LabelEncoder()

* Birden fazla label encoding dönüşümünü yapmak için bir sınıf oluşturdum bu sınıf içinde labelEncoding fonksiyonu parametre olarak gelen değişken listesindeki bütün değişkenlere label encoding işlemi yapar.

In [ ]:
class ConvertingCategoricalValues():

    def labelEncoding(self,columns):
        for column in columns:
            df[f"{column}"]=label_encoder.fit_transform(df[f"{column}"])
        
instance=ConvertingCategoricalValues()
instance.labelEncoding(["Bluetooth", "CiftHat", "4G", "3G", "Dokunmatik", "WiFi"])
df.head()


#### Dummy Variable Oluşturma "get_dummies()"

Fiyat aralığını geçici olarak veri setinden çıkartıyoruz. 

In [ ]:
fiyatlar = df["FiyatAraligi"]
fiyatlar

In [ ]:
df = df.drop("FiyatAraligi", axis=1)

df

In [ ]:
renkler = pd.get_dummies(df["Renk"], prefix="Renk").astype(int)

renkler

In [ ]:
df = pd.concat([df.drop("Renk", axis=1), renkler], axis=1)


In [ ]:
df = pd.concat([df, fiyatlar], axis=1)


In [ ]:
df


#### Önemli Not : 
Veri çerçevesinde aynı değeri ifade eden pek çok değişken oluştu. Bu değişkenler aynı şeyi ifade ettiği için bir makine öğrenmesi modeli kurulduğunda yanıltıcı olacaktır. Aynı şeyi ifade eden değişkenleri silelim. 

**FiyatAraligi ve Renk** değişkenleri silinmelidir çünkü zaten bu değişkenlere ait dummy variables ürettik. 

Ayrıca üretilen FiyatAraligi ve Renk değişkenlerinden birisi de silinmelidir. Mesela **FiyatAraligi_Normal, FiyatAraligi_Pahalı ve FiyatAraligi_Ucuz** değişkenlerine bakarak **FiyatAraligi_ÇokUcuz**'ün değeri hakkında çıkarımda bulunabiliyoruz. Yani aslında üretilen değişken sayısnın 1 eksiği değişken değerini görmek model için yeterli olacaktır. 

O halde FiyatAraligi ve FiyatAraligi_(Normal,Pahalı,Ucuz,ÇokUcuz) dummy değişkenlerinden birisi makine öğrenmesi modeli kurmadan önce silinmelidir. Aynı işlem Renk değişkeni ve dummy değişkenleri için de geçerlidir. Biz şimdilik silmiyoruz, silmek istesek aşağıdaki komut çalıştırılabilir.

df.drop(["FiyatAraligi", "FiyatAraligi_ÇokUcuz"], axis = 1, inplace = True

(**FiyatAraligi_ÇokUcuz değişkenini silmemin özel bir anlamı yok, herhangi biri silinebilir.**)

# Veri Eğitimi


Veri setinden 'FiyatAraligi' adlı sütunu çıkararak bağımsız değişkenleri (X) ve sadece 'FiyatAraligi' sütununu hedef değişken olarak seçerek (y), makine öğrenimi modelinin eğitim için gerekli girdileri ve çıktıları hazırlıyoruz. Bu ayrım, modelin bağımsız değişkenlerden hedef değişkeni tahminlemek için eğitilmesini sağlar.

In [ ]:
X = df.drop('FiyatAraligi', axis=1)  
y = df['FiyatAraligi'] 

X.head() komutu, bağımsız değişkenleri (X) içeren veri setinin ilk beş satırını görüntüler. Bu, veri setinin ön izlemesini sağlar ve makine öğrenimi modeli için kullanılan girdi verilerinin nasıl göründüğünü kontrol etmemizi sağlar.

In [ ]:
X.head()


y.head() komutu, hedef değişkeni (y) içeren veri setinin ilk beş satırını görüntüler. Bu, hedef değişkenin ilk beş gözlemini gösterir ve makine öğrenimi modelinin eğitiminde kullanılan çıktı verilerinin nasıl dağıldığını kontrol etmemizi sağlar

In [ ]:
y.head()

veri setini eğitim ve test kümelerine ayırmak için kullanılır. Veri seti, genellikle bir kısmı modelin eğitimi için (eğitim kümesi) ve geri kalanı modelin performansının değerlendirilmesi için (test kümesi) bölmek için kullanılır.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)


X_train değişkeni, modelin eğitiminde kullanılan girdi verilerini temsil eder.

In [ ]:
X_train

X_test değişkeni, modelin eğitim sonrası performansını değerlendirmek için kullanılan girdi verilerini temsil eder.

In [ ]:
X_test

y_train değişkeni, modelin eğitiminde kullanılan çıktı verilerini temsil eder.

In [ ]:
y_train

y_test değişkeni, modelin eğitim sonrası performansını değerlendirmek için kullanılan çıktı verilerini temsil eder.

In [ ]:
y_test

# GaussianNB Modeli


model_GNB = GaussianNB() kodu, Gaussian Naive Bayes sınıflandırma modelini oluşturur. Bu model, sınıflandırma problemleri için kullanılır ve özellikler arasındaki bağımsızlık varsayımını yapar. Model, veri setindeki desenleri öğrenmek için eğitilir ve ardından tahminler yapabilir

In [ ]:
model_GNB = GaussianNB()


model_GNB.fit(X_train, y_train) kodu, Gaussian Naive Bayes modelini eğitmek için kullanılır. Bu yöntem, eğitim verilerini (bağımsız değişkenler X_train ve hedef değişken y_train) kullanarak modelin iç parametrelerini ve istatistiklerini öğrenir.

In [ ]:
model_GNB.fit(X_train, y_train)



y_pred = model_GNB.predict(X_test) kodu, eğitilmiş Gaussian Naive Bayes modelini kullanarak test veri kümesi (X_test) üzerinde tahminler yapar. Model, test veri kümesindeki bağımsız değişkenlerin (X_test) kullanarak hedef değişkenin (y_test) tahminlenmesini gerçekleştirir.

In [ ]:
y_pred = model_GNB.predict(X_test)
y_pred

 gerçek ve model tarafından tahmin edilen fiyatları içeren iki ayrı DataFrame oluşturur ve bunları birleştirir. Böylece, her bir gözlem için gerçek ve tahmini fiyatları içeren bir DataFrame elde edilir.

In [ ]:
fiyatlar = pd.DataFrame(y_test)

tahminler = pd.DataFrame(y_pred,columns=["Tahmini Fiyatlar"])

merged_df = pd.merge(fiyatlar, tahminler, left_index=True, right_index=True, suffixes=('_Gerçek', '_Tahmini'))
merged_df


GNB_cm değişkeni, bu karışıklık matrisini içerir ve modelin performansını değerlendirmek için kullanılabilir.

In [ ]:
GNB_cm = confusion_matrix(y_test, y_pred)

GNB_cm


GNB_accscore değişkeni, bu doğruluk skorunu içerir ve modelin test veri kümesinde ne kadar doğru tahmin yaptığını gösterir. Yüksek bir doğruluk skoru, modelin veri setindeki desenleri doğru bir şekilde tanıdığını ve tahmin ettiğini gösterir.

In [ ]:
GNB_accscore = accuracy_score(y_test, y_pred)
GNB_accscore

precision_score(y_test, y_pred, average='weighted') kodu, Gaussian Naive Bayes modelinin sınıflandırma performansını değerlendirmek için kullanılan hassasiyet (precision) skorunu hesaplar. Hassasiyet, modelin pozitif olarak tahmin ettiği örneklerin ne kadarının gerçekten pozitif olduğunu gösterir.

In [ ]:
GNB_precision = precision_score(y_test, y_pred, average='weighted')
GNB_precision

recall_score(y_test, y_pred, average='weighted') kodu, Gaussian Naive Bayes modelinin sınıflandırma performansını değerlendirmek için kullanılan duyarlılık (recall) skorunu hesaplar. Duyarlılık, gerçek pozitiflerin (TP) tespit edilme oranını ifade eder.

In [ ]:
GNB_recall = recall_score(y_test, y_pred, average='weighted')
GNB_recall

Gaussian Naive Bayes modelinin test veri kümesindeki ağırlıklı F1 skorunu hesaplar. Ağırlıklı F1 skoru, modelin farklı sınıflar için hem hassasiyeti hem de duyarlılığı dikkate alarak ne kadar iyi performans gösterdiğini gösterir.

In [ ]:
GNB_f1 = f1_score(y_test, y_pred, average='weighted')
GNB_f1

modelin sınıflandırma performansını değerlendiren bir rapor oluşturur. Rapor, her bir sınıf için hassasiyet (precision), duyarlılık (recall), F1 skoru ve destek sayısını içerir.

performans_sonuc değişkeni, classification_report() fonksiyonunun çıktısını içerir. Bu çıktı, modelin test veri kümesindeki sınıflandırma performansını ayrıntılı bir şekilde gösterir.

In [ ]:
performans_sonuc = classification_report(y_test, y_pred)

print("Model Performansı:")
print(performans_sonuc)

# KNN Modeli

K-En Yakın Komşu (KNN) sınıflandırıcı modelini oluşturur. KNN, gözlemleri sınıflandırmak için kullanılan basit ve etkili bir makine öğrenimi algoritmasıdır.

n_neighbors=30 parametresi, KNN algoritmasında komşu sayısını belirtir. Bu durumda, algoritma her bir gözlem için en yakın 30 komşuyu dikkate alacaktır.

model_Knn değişkeni, oluşturulan KNN sınıflandırıcı modelini temsil eder ve veri setindeki desenleri tanımak ve sınıflandırmak için kullanılır. Bu model, eğitildikten sonra yeni verilere uygulanabilir ve sınıflandırma tahminleri yapabilir.

In [ ]:
model_Knn = KNeighborsClassifier(n_neighbors=30)

K-En Yakın Komşu (KNN) sınıflandırıcı modelini eğitmek için kullanılır. X_train ve y_train değişkenleri, modelin eğitimi için kullanılacak olan bağımsız değişkenler ve hedef değişkeni içerir.

Model, eğitim verilerine (X_train ve y_train) dayanarak öğrenir ve her bir gözlem için en yakın komşularını belirler. Bu, modelin sınıflandırma kararlarını yapabilmesi için gerekli olan desenleri tanımasını sağlar.

Yani, model_Knn.fit(X_train , y_train) kodu, KNN modelini eğitir ve modelin eğitildikten sonra yeni verilere uygulanabilir hale gelmesini sağlar.

In [ ]:
model_Knn.fit(X_train , y_train)

bu kod sonucunda elde edilen y_pred dizisi, test veri kümesindeki her bir gözlem için model tarafından yapılan sınıflandırma tahminlerini içerir. Bu tahminler, test edilen verilerin model tarafından sınıflandırılması sonucu elde edilir.

In [ ]:
y_pred = model_Knn.predict(X_test)
y_pred

y_test, test veri kümesindeki gerçek sınıf etiketlerini içeren bir veri dizisidir. Bu veri dizisi, modelin performansını değerlendirmek için kullanılan tahminlerle karşılaştırılır. Görselleştirme, modelin gerçek sınıfları ne kadar doğru tahmin ettiğini belirlemek için önemlidir. Bu veri dizisinin içeriği, modelin test veri kümesindeki gerçek sınıfları doğru bir şekilde tahmin edip etmediğini belirlemek için kullanılabilir.

In [ ]:
y_test

K-En Yakın Komşu (KNN) modelinin gerçek ve tahmini fiyatları karşılaştırmak için iki ayrı DataFrame oluşturur ve bunları birleştirir.

İlk olarak, pd.DataFrame(y_test) kodu, gerçek fiyatları içeren bir DataFrame oluşturur ve fiyatlar değişkenine atar.
Ardından, pd.DataFrame(y_pred,columns=["Tahmini Fiyatlar"]) kodu, model tarafından yapılan tahminleri içeren bir DataFrame oluşturur ve bu DataFrame'in sütun adını "Tahmini Fiyatlar" olarak belirler. Bu DataFrame tahminler değişkenine atılır.
Son olarak, pd.merge() fonksiyonu, fiyatlar ve tahminler DataFrame'lerini birleştirir. left_index=True ve right_index=True parametreleri, birleştirme işleminin dizinler (index) üzerinden yapılacağını belirtir. suffixes=('_Gerçek', '_Tahmini') parametresi ise, birleştirilen sütunların isimlerine eklenecek ek sonekleri belirtir.
Bu işlemin sonucunda oluşan Knn_merged, gerçek ve tahmini fiyatları içeren bir DataFrame olur. Bu DataFrame, her bir gözlem için gerçek ve tahmini fiyatları içerir.

In [ ]:
fiyatlar = pd.DataFrame(y_test)

tahminler = pd.DataFrame(y_pred,columns=["Tahmini Fiyatlar"])

Knn_merged = pd.merge(fiyatlar, tahminler, left_index=True, right_index=True, suffixes=('_Gerçek', '_Tahmini'))
Knn_merged


confusion_matrix(y_test, y_pred) kodu, K-En Yakın Komşu (KNN) modelinin performansını değerlendirmek için kullanılan bir karışıklık matrisi (confusion matrix) oluşturur. Bu matris, modelin gerçek ve tahmini sınıflarını karşılaştırır ve sınıflandırma performansını görsel olarak gösterir.

In [ ]:
KNN_cm = confusion_matrix(y_test, y_pred)

KNN_cm

accuracy_score(y_test, y_pred) kodu, K-En Yakın Komşu (KNN) modelinin sınıflandırma performansını değerlendirmek için kullanılan doğruluk (accuracy) skorunu hesaplar. Bu skor, modelin doğru tahminlerinin toplam gözlem sayısına oranını ifade eder ve genellikle sınıflandırma modellerinin performansını ölçmek için kullanılan temel bir metriktir.

KNN_accscore değişkeni, bu doğruluk skorunu içerir ve modelin test veri kümesinde ne kadar doğru tahmin yaptığını gösterir. Yüksek bir doğruluk skoru, modelin test veri kümesindeki sınıflandırma performansının iyi olduğunu gösterir.

In [ ]:
KNN_accscore = accuracy_score(y_test, y_pred)
KNN_accscore

precision_score(y_test, y_pred, average='weighted') kodu, K-En Yakın Komşu (KNN) modelinin sınıflandırma performansını değerlendirmek için kullanılan hassasiyet (precision) skorunu hesaplar. Hassasiyet, modelin pozitif olarak tahmin ettiği örneklerin ne kadarının gerçekten pozitif olduğunu gösterir.

Bu kod, ortalama 'weighted' parametresiyle hesaplanan ağırlıklı bir hassasiyet skoru döndürür. Ağırlıklı hassasiyet, her bir sınıfın desteğine (gerçek örnek sayısı) göre ağırlıklandırılmış hassasiyetlerin ortalamasını alır.

KNN_precision değişkeni, bu hassasiyet skorunu içerir ve modelin test veri kümesindeki sınıflandırma performansını ölçer. Ağırlıklı hassasiyet skoru, modelin farklı sınıflar için ne kadar iyi performans gösterdiğini gösterir.

In [ ]:
KNN_precision = precision_score(y_test, y_pred, average='weighted')
KNN_precision

KNN_f1 değişkeni, bu F1 skorunu içerir ve modelin test veri kümesindeki sınıflandırma performansını ölçer. Ağırlıklı F1 skoru, modelin farklı sınıflar için hem hassasiyeti hem de duyarlılığı dikkate alarak ne kadar iyi performans gösterdiğini gösterir.

In [ ]:
KNN_f1 = f1_score(y_test, y_pred, average='weighted')
KNN_f1

KNN_recall değişkeni, bu duyarlılık skorunu içerir ve modelin test veri kümesindeki sınıflandırma performansını ölçer. Ağırlıklı duyarlılık skoru, modelin farklı sınıflar için ne kadar iyi gerçek pozitifler tespit ettiğini gösterir

In [ ]:
KNN_recall = recall_score(y_test, y_pred, average='weighted')
KNN_recall

sınıflandırma modelinin performansını ayrıntılı bir rapor olarak yazdırır. Rapor, her bir sınıf için hassasiyet (precision), duyarlılık (recall), F1 skoru ve destek sayısını içerir. Bu rapor, modelin sınıflandırma performansını değerlendirmek için kullanılabilir ve farklı sınıflar arasındaki performans farklarını gösterir.

In [ ]:
print(metrics.classification_report(y_test, y_pred))


K-En Yakın Komşu (KNN) modelinin performansını farklı komşu sayıları kullanarak değerlendirmek için bir döngü oluşturur. Her bir komşu sayısı için modelin doğruluk (accuracy), hassasiyet (precision), duyarlılık (recall) ve F1 skoru hesaplanır ve bu skorlar ilgili listelere eklenir. Bu şekilde, farklı komşu sayılarının model performansına etkisi karşılaştırılabilir.

In [ ]:
komsu_sayilari = list(range(2,51))
skorlar_accuracy = []
skorlar_precision = []
skorlar_recall = []
f1_scores = list()



Bu döngü, farklı komşu sayıları için K-En Yakın Komşu (KNN) modelleri oluşturur ve her bir modelin performansını değerlendirir. Her bir komşu sayısı için modelin doğruluk (accuracy), hassasiyet (precision), duyarlılık (recall) ve F1 skoru hesaplanır. Bu skorlar ilgili listelere eklenir, böylece farklı komşu sayılarının model performansına etkisi karşılaştırılabilir

In [ ]:
for komsu in komsu_sayilari : 
        Knn = KNeighborsClassifier(n_neighbors = komsu)
        Knn.fit(X_train, y_train)
        y_pred_Knn = Knn.predict(X_test)
        skorlar_ac = accuracy_score(y_test, y_pred_Knn)
        skorlar_pre = precision_score(y_test, y_pred_Knn, average='weighted')
        skorlar_rec = recall_score(y_test, y_pred_Knn, average='weighted')
        skorlar_f1 = f1_score(y_test, y_pred_Knn, average='weighted')
        
        skorlar_accuracy.append(skorlar_ac)
        skorlar_precision.append(skorlar_pre)
        skorlar_recall.append(skorlar_rec)


 farklı komşu sayılarına karşılık gelen K-En Yakın Komşu (KNN) modellerinin doğruluk (accuracy) skorlarını görselleştirmek için bir çizgi grafiği oluşturur. Grafikte, x-ekseni komşu sayılarını (K) ve y-ekseni doğruluk skorlarını temsil eder. Böylece, komşu sayısının doğruluk skoru üzerindeki etkisi görsel olarak gösterilir.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(komsu_sayilari, skorlar_accuracy, marker='o')
plt.xlabel('Komşu Sayısı (K)')
plt.ylabel('Accuracy Doğruluk Skoru')
plt.title('KNN Modelinin Komşu Sayısına Göre Accuracy Doğruluk Skoru')
plt.grid(True)
plt.xticks(komsu_sayilari)
plt.show()


 farklı komşu sayılarına karşılık gelen K-En Yakın Komşu (KNN) modellerinin hassasiyet (precision) skorlarını görselleştirmek için bir çizgi grafiği oluşturur. Grafikte, x-ekseni komşu sayılarını (K) ve y-ekseni hassasiyet skorlarını temsil eder. Böylece, komşu sayısının hassasiyet skoru üzerindeki etkisi görsel olarak gösterilir.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(komsu_sayilari, skorlar_precision, marker='o')
plt.xlabel('Komşu Sayısı (K)')
plt.ylabel('Precision Doğruluk Skoru')
plt.title('KNN Modelinin Komşu Sayısına Göre Precision Doğruluk Skoru')
plt.grid(True)
plt.xticks(komsu_sayilari)
plt.show()

 farklı komşu sayılarına karşılık gelen K-En Yakın Komşu (KNN) modellerinin duyarlılık (recall) skorlarını görselleştirmek için bir çizgi grafiği oluşturur. Grafikte, x-ekseni komşu sayılarını (K) ve y-ekseni duyarlılık skorlarını temsil eder. Böylece, komşu sayısının duyarlılık skoru üzerindeki etkisi görsel olarak gösterilir.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(komsu_sayilari,skorlar_recall, marker='o')
plt.xlabel('Komşu Sayısı (K)')
plt.ylabel('Recall Doğruluk Skoru')
plt.title('KNN Modelinin Komşu Sayısına Göre Recall Doğruluk Skoru')
plt.grid(True)
plt.xticks(komsu_sayilari)
plt.show()

# Decision Tree

fit() fonksiyonu kullanılarak model eğitilir. X_train ve y_train değişkenleri, eğitim verilerini içerir. Bu adımlardan sonra, DecTree adlı değişken eğitilmiş karar ağacı modelini temsil eder.






In [ ]:
# Decision Tree Modeli (criterion = "gini")
DecTree = DecisionTreeClassifier(random_state = 42, criterion='gini')
DecTree = DecTree.fit(X_train, y_train)

X_test değişkeni, test veri kümesini temsil eder. y_pred değişkeni, model tarafından yapılan tahminleri içerir ve her bir gözlem için tahmin edilen sınıf etiketlerini içerir.

In [ ]:
y_pred = DecTree.predict(X_test)

y_pred değişkeni, eğitilmiş Karar Ağacı (Decision Tree) modeli kullanılarak yapılan tahminleri içerir. Her bir örnek için model tarafından yapılan sınıflandırma tahminlerini içerir. Bu tahminler, test veri kümesindeki her bir gözlem için modelin sınıflandırma kararlarını temsil eder.

In [ ]:
y_pred

DT_Gini_cm değişkeni, bu karışıklık matrisini içerir ve modelin performansını değerlendirmek için kullanılabilir.

In [ ]:
DT_Gini_cm = confusion_matrix(y_test, y_pred)
DT_Gini_cm

DT_gini_accscore değişkeni, bu doğruluk skorunu içerir ve modelin test veri kümesinde ne kadar doğru tahmin yaptığını gösterir. Yüksek bir doğruluk skoru, modelin test veri kümesindeki sınıflandırma performansının iyi olduğunu gösterir.

In [ ]:
DT_gini_accscore = accuracy_score(y_test, y_pred)
DT_gini_accscore

In [ ]:
DT_gini_precision = precision_score(y_test, y_pred, average='weighted')
DT_gini_precision 

****

DT_gini_f1 değişkeni, bu F1 skorunu içerir ve modelin test veri kümesindeki sınıflandırma performansını ölçer. Ağırlıklı F1 skoru, modelin farklı sınıflar için hem hassasiyeti hem de duyarlılığı dikkate alarak ne kadar iyi performans gösterdiğini gösterir.

In [ ]:
DT_gini_f1 = f1_score(y_test, y_pred, average='weighted')
DT_gini_f1

DT_gini_recall değişkeni, bu duyarlılık skorunu içerir ve modelin test veri kümesindeki sınıflandırma performansını ölçer. Ağırlıklı duyarlılık skoru, modelin farklı sınıflar için ne kadar iyi gerçek pozitifler tespit ettiğini gösterir.

In [ ]:
DT_gini_recall = recall_score(y_test, y_pred, average='weighted')
DT_gini_recall

Karar Ağacı (Decision Tree) modelini yeniden oluşturur, ancak bu sefer kriter olarak "entropy" kullanılır.

DecisionTreeClassifier sınıfı, karar ağacı modelini oluşturmak için kullanılır. random_state = 42 parametresi, modelin tekrarlanabilirliğini sağlamak için kullanılır. criterion='entropy' parametresi, karar ağacının bölünme noktalarını belirlemek için kullanılan ölçütü belirtir; burada "entropy" ölçütü kullanılmıştır.

In [ ]:
# Decision Tree Modeli (criterion = "entropy")
DecTree = DecisionTreeClassifier(random_state = 42, criterion='entropy')
DecTree = DecTree.fit(X_train, y_train)

X_test değişkeni, test veri kümesini temsil eder. y_pred değişkeni, model tarafından yapılan tahminleri içerir ve her bir gözlem için tahmin edilen sınıf etiketlerini içerir. Bu tahminler, test veri kümesindeki her bir gözlem için modelin sınıflandırma kararlarını temsil eder.

In [ ]:
y_pred = DecTree.predict(X_test)

y_pred değişkeni, eğitilmiş Karar Ağacı (Decision Tree) modeli kullanılarak yapılan tahminleri içerir. Her bir örnek için model tarafından yapılan sınıflandırma tahminlerini içerir. Bu tahminler, test veri kümesindeki her bir gözlem için modelin sınıflandırma kararlarını temsil eder.

In [ ]:
y_pred

DT_entropy_cm değişkeni, bu karışıklık matrisini içerir ve modelin performansını değerlendirmek için kullanılabilir.

In [ ]:
DT_entropy_cm = confusion_matrix(y_test, y_pred)
DT_entropy_cm

DT_entropy_accscore değişkeni, bu doğruluk skorunu içerir ve modelin test veri kümesinde ne kadar doğru tahmin yaptığını gösterir. Yüksek bir doğruluk skoru, modelin test veri kümesindeki sınıflandırma performansının iyi olduğunu gösterir.

In [ ]:
DT_entropy_accscore = accuracy_score(y_test, y_pred)
DT_entropy_accscore

DT_entropy_precision değişkeni, bu hassasiyet skorunu içerir ve modelin test veri kümesindeki sınıflandırma performansını ölçer. Ağırlıklı hassasiyet skoru, modelin farklı sınıflar için ne kadar iyi performans gösterdiğini gösterir.

In [ ]:
DT_entropy_precision = precision_score(y_test, y_pred, average='weighted')
DT_entropy_precision 

DT_entropy_f1 değişkeni, bu F1 skorunu içerir ve modelin test veri kümesindeki sınıflandırma performansını ölçer. Ağırlıklı F1 skoru, modelin farklı sınıflar için hem hassasiyeti hem de duyarlılığı dikkate alarak ne kadar iyi performans gösterdiğini gösterir.

In [ ]:
DT_entropy_f1 = f1_score(y_test, y_pred, average='weighted')
DT_entropy_f1 

DT_entropy_recall değişkeni, bu duyarlılık skorunu içerir ve modelin test veri kümesindeki sınıflandırma performansını ölçer. Ağırlıklı duyarlılık skoru, modelin farklı sınıflar için ne kadar iyi gerçek pozitifler tespit ettiğini gösterir.

In [ ]:
DT_entropy_recall = recall_score(y_test, y_pred, average='weighted')
DT_entropy_recall

 karar ağacı (Decision Tree) modellerinin performans ölçümlerini içeren listeleri oluşturur. Her bir model için doğruluk (accuracy), hassasiyet (precision), F1 skoru ve duyarlılık (recall) ölçümleri, karışıklık matrisi ve sınıflandırma raporları kullanılarak hesaplanmıştır.

accscores_DT, precision_DT, f1_DT ve recall_DT listeleri, sırasıyla doğruluk, hassasiyet, F1 skoru ve duyarlılık ölçümlerini içerir. models_DT listesi, her bir model için isimleri içerir.

In [ ]:
accscores_DT = [DT_gini_accscore, DT_entropy_accscore]
precision_DT = [DT_gini_accscore, DT_entropy_accscore]
f1_DT = [DT_gini_accscore, DT_entropy_accscore]
recall_DT = [DT_gini_accscore, DT_entropy_accscore]
models_DT = ['Decision Tree (gini)', 'Decision Tree (entropy)']


models_DT değişkeni, her bir modelin adını içerir. accscores_DT değişkeni, her bir model için doğruluk skorunu içerir.

Grafik, karar ağaçlarının (gini ve entropy kriterleri ile eğitilmiş) doğruluk skorlarını karşılaştırır. Görsel olarak, hangi modelin daha iyi performans gösterdiğini görmek için kullanılabilir.

In [ ]:
plt.bar(models_DT, accscores_DT, color=['orange', 'red'])
plt.xlabel('Modeller')
plt.ylabel('Accuracy Başarı Skoru')
plt.title('KNN ve GNB modellerinin Başarı Karşılaştırması')
plt.ylim(0, 1)  
plt.show()


In [ ]:
plt.bar(models_DT, precision_DT, color=['blue', 'pink'])
plt.xlabel('Modeller')
plt.ylabel('Precision Başarı Skoru')
plt.title('KNN ve GNB modellerinin Başarı Karşılaştırması')
plt.ylim(0, 1)  
plt.show()

models_DT değişkeni, her bir modelin adını içerir. accscores_DT değişkeni, her bir model için doğruluk skorunu içerir.

Grafik, karar ağaçlarının (gini ve entropy kriterleri ile eğitilmiş) doğruluk skorlarını karşılaştırır. Görsel olarak, hangi modelin daha iyi performans gösterdiğini görmek için kullanılabilir.

In [ ]:
plt.bar(models_DT, f1_DT, color=['yellow', 'green'])
plt.xlabel('Modeller')
plt.ylabel('Accuracy Başarı Skoru')
plt.title('KNN ve GNB modellerinin Başarı Karşılaştırması')
plt.ylim(0, 1)  
plt.show()


models_DT değişkeni, her bir modelin adını içerir. f1_DT değişkeni, her bir model için F1 skorunu içerir.

Grafik, karar ağaçlarının (gini ve entropy kriterleri ile eğitilmiş) F1 skorlarını karşılaştırır. Görsel olarak, hangi modelin daha iyi performans gösterdiğini görmek için kullanılabilir.

In [ ]:
plt.bar(models_DT, recall_DT, color=['blue', 'red'])
plt.xlabel('Modeller')
plt.ylabel('Accuracy Başarı Skoru')
plt.title('KNN ve GNB modellerinin Başarı Karşılaştırması')
plt.ylim(0, 1)  
plt.show()

# MODELLERİN BAŞARI KARŞILAŞTIRMASI

models_DT değişkeni, her bir modelin adını içerir. recall_DT değişkeni, her bir model için duyarlılık skorunu içerir.

Grafik, karar ağaçlarının (gini ve entropy kriterleri ile eğitilmiş) duyarlılık skorlarını karşılaştırır. Görsel olarak, hangi modelin daha iyi performans gösterdiğini görmek için kullanılabilir.

In [ ]:
scores = [KNN_accscore,DT_gini_accscore,GNB_accscore]
models = ['KNN', 'DCT','GNB']

# Create the bar plot
plt.bar(models, scores, color=['blue', 'green','orange'])
plt.xlabel('Modeller')
plt.ylabel('Başarı Skoru')
plt.title('KNN, DCT,GNB modellerinin Başarı Karşılaştırması')
plt.ylim(0, 1)  
plt.show()